In [18]:
import wandb
import pandas as pd
wandb.login()
%env "WANDB_NOTEBOOK_NAME" "DaNet"

env: "WANDB_NOTEBOOK_NAME"="DaNet"


In [19]:
import time
def exec_time(start, end):
  diff_time = end - start
  m, s = divmod(diff_time, 60)
  h, m = divmod(m, 60)
  s,m,h = (round(s, 2)), (round(m, 2)), (round(h, 2))
  if h > 0:
    return("{0:.2f} Hrs {1:.2f} Mins {2:.2f} Secs".format(h, m, s))
  elif h==0 and m > 0:
    return("{0:.2f} Mins {1:.2f} Secs".format(m, s))
  else:
    return("{0:.2f} Secs".format(s))

In [20]:
run = wandb.init()

In [21]:
start = time.time()
!python demo.py --checkpoint=data/pretrained_model/danet_model_h36m_itw.pt --img_dir ./examples --use_opendr
end = time.time()

/home/abhishek/Desktop/research/DaNet-3DHumanReconstruction/models/core/config.py:1070: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  yaml_cfg = AttrDict(yaml.load(f))
using decomposed predictor.
input mode: IUV
Demo results have been saved in ./output.


In [22]:
run.finish()

In [23]:
api = wandb.Api() 
run = api.run(f"sisodiya2421/DaNet-3DHumanReconstruction/{run.name}/{run.id}")
system_metrics = run.history(stream = 'events')
system_metrics.to_csv("DaNet_metrics.csv")

In [24]:
system_metrics

,system.network.sent,system.network.recv,system.disk,_wandb,system.gpu.0.temp,system.gpu.0.memory,system.gpu.0.gpu,_runtime,system.proc.memory.rssMB,system.proc.memory.availableMB,system.cpu,system.proc.cpu.threads,system.memory,system.proc.memory.percent,system.gpu.0.memoryAllocated,_timestamp
0,25888,32443,63,True,50,0,0,10,102.92,2995.65,23.23,11,61.73,1.32,0.36,1653073172
1,86709,110467,63,True,57,18,40,17,104.14,3098.91,25.00,12,60.40,1.33,0.36,1653073179


In [25]:
temp = system_metrics.copy()

In [26]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

In [27]:
fig = make_subplots(rows=1, cols=2, start_cell="top-left", 
                    subplot_titles=("GPU Utilization (%)", 
                                    "GPU Memory Allocated (%)"))

fig.add_trace(go.Scatter(x=temp["_runtime"], y=temp["system.gpu.0.gpu"]), row=1, col=1)
fig.add_trace(go.Scatter(x=temp["_runtime"], y=temp["system.gpu.0.memoryAllocated"]), row=1, col=2)

fig.update_xaxes(title_text="Time (seconds)", row=1, col=1)
fig.update_xaxes(title_text="Time (seconds)",  row=1, col=2)

fig.update_yaxes(range=[-2, 100],row=1, col=1)
fig.update_yaxes(range=[-2, 100], row=1, col=2)


fig.update_layout(title_text="GPU Usage Stats", showlegend=False,)

fig.show()

In [28]:
fig = make_subplots(rows=1, cols=2, start_cell="top-left", 
                    subplot_titles=("CPU Utilization (%)", 
                                    "System Memory Utilization (%)"))

fig.add_trace(go.Scatter(x=temp["_runtime"], 
                         y=temp["system.cpu"]), row=1, col=1)
fig.add_trace(go.Scatter(x=temp["_runtime"], y=temp["system.memory"]), row=1, col=2)

fig.update_xaxes(title_text="Time (seconds)", row=1, col=1)
fig.update_xaxes(title_text="Time (seconds)",  row=1, col=2)

fig.update_yaxes(range=[-2, 100],row=1, col=1)
fig.update_yaxes(range=[-2, 100], row=1, col=2)


fig.update_layout(title_text="CPU and Memory Usage Stats", showlegend=False,)

fig.show()

In [31]:
data = {
    'Resources':['Time taken to Generate 3D body', 
                 'GPU time spent accessing Memory', 
                 'GPU Memory Allocated', 
                 'System Memory Utilised',
                 'CPU Utilisation'],
    'Utilisation':[str(exec_time(start,end)), 
                   str(system_metrics["system.gpu.0.memory"].max()) + " Secs", 
                   str(system_metrics["system.gpu.0.memoryAllocated"].max()) + " %", 
                   str(system_metrics["system.memory"].max()) + " %",
                   str(system_metrics["system.cpu"].max()) + " %"]}
df = pd.DataFrame(data)
df

,Resources,Utilisation
0,Time taken to Generate 3D body,7.64 Secs
1,GPU time spent accessing Memory,18 Secs
2,GPU Memory Allocated,0.36 %
3,System Memory Utilised,61.73 %
4,CPU Utilisation,25.0 %


In [ ]:
data = {
    'Resources':['Time taken to Generate 3D body', 
                 'GPU time spent accessing Memory', 
                 'GPU Memory Allocated', 
                 'System Memory Utilised',
                 'CPU Utilisation'],
    'Utilisation':[str(exec_time(start,end)), 
                   str(system_metrics["system.gpu.0.memory"].max()) + " Secs", 
                   str(system_metrics["system.gpu.0.memoryAllocated"].max()) + " %", 
                   str(system_metrics["system.memory"].max()) + " %",
                   str(system_metrics["system.cpu"].max()) + " %"]}
df = pd.DataFrame(data)
df

,Resources,Utilisation
0,Time taken to Generate 3D body,8.05 Secs
1,GPU time spent accessing Memory,12.0 Secs
2,GPU Memory Allocated,15.24 %
3,System Memory Utilised,70.85 %
4,CPU Utilisation,26.3 %
